# Parte 1. Obtención de datos

El objetivo de este notebook es realizar una descarga iterativa de algunos productos que hacen parte de las categorías del Marketplace de Mercadolibre. 
Para llevar acabo este proceso, se tuvo en cuenta cada una de las 32 categorías en la lista de productos, junto con sus respectivas subcategorías. Se realizó una extracción inicial de 50 productos por subcategoría para un total de 19271 productos.

Luego de una exploración inicial, se define que los siguientes son los atributos de interés para el problema a resolver: 
1. Categoría 
2. Subcategoría
3. Nombre del producto 
4. Marca
5. Precio 
6. Imagen 

## Carga de datos

In [1]:
import pandas as pd
import numpy as np
import regex,requests,unidecode,os, glob
from pathlib import Path  
import cv2
import plotly.express as px
import matplotlib.pyplot as plt 
from tqdm import tqdm
import shutil, sys
import urllib.request
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk import FreqDist
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn import metrics


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adelaidazuluaga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adelaidazuluaga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Lista de categorías de productos

In [10]:
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
prod_categories= cats.json()

### Obtener sub categorías por cada categoría

In [3]:
def obtain_sub_categories(general_category:str): 
    try:
        url = f'https://api.mercadolibre.com/sites/MLA/search?category={general_category}'
        request = requests.get(url)
        items = request.json()
        sub_categories_names = [i['name'] for i in items['available_filters'][0]['values']]
        sub_categories_ids = [i['id'] for i in items['available_filters'][0]['values']]
        df_sub_cat = pd.DataFrame(list(zip(sub_categories_ids, sub_categories_names)),columns=['id','Name'])
    except Exception as e:
        print (e.message, e.args)
    return df_sub_cat

### Descargar iterativamente la información de los productos pertenecientes a cada categoría

In [4]:
def obtain_json_items_iteratively(df_sub_cat,offset_list):
    offsets= list(range(0,50,50)) #50 ELEMENTOS POR SUBCATEGORIA
    results = []
    sub_cat_list = []
    offsets = offset_list
    sub_cat_id = list(df_sub_cat['id'])
    sub_cat_name = list(df_sub_cat['Name'])
    for c in sub_cat_id:
        url = f'https://api.mercadolibre.com/sites/MLA/search?category={c}&offset={offsets[0]}'
        request = requests.get(url)
        data = request.json()
        for element in data['results']:
            results.append(element)
            sub_cat_list.append(list(df_sub_cat[df_sub_cat['id']==c]['Name'])[0])

    return results,sub_cat_list

### Extrer la marca del producto  

In [5]:
def get_product_brand(items_cat):
    brands = []
    price = []
    for att in items_cat['attributes']:
        find_attr = att['id']
        if 'BRAND' in find_attr or 'Brand' in find_attr:
            brands.append(att['value_name'])
            break
    return brands

### Limpieza del texto (NLP preprocessing) 

In [6]:
def pre_proc_nlp(text_df):
    cleaned_df =[]
    for text in text_df:
        clean_text = regex.sub(r'\&[a-z]+\;', '', text)
        clean_text = regex.sub(r"\n", " ", text)
        clean_text = text.lower()
        clean_text = unidecode.unidecode(clean_text)
        clean_text = regex.sub(r'\s+', ' ', clean_text)
        clean_text = regex.sub('[^a-zA-Z]', ' ', clean_text).strip(' ')
        cleaned_df.append(clean_text)
    return cleaned_df

In [7]:
def clean_name(text_name):
    text = text_name
    clean_text = regex.sub(r'\&[a-z]+\;', '', text)
    clean_text = regex.sub(r"\n", " ", text)
    clean_text = text.lower()
    clean_text = unidecode.unidecode(clean_text)
    clean_text = regex.sub(r'\s+', ' ', clean_text)
    clean_text = regex.sub('[^a-zA-Z]', ' ', clean_text).strip(' ')
    cleaned_df = clean_text
    return cleaned_df

### Crear dataframe con la información de los productos

In [8]:
def create_pd_product(item_json,item_category, item_sub_cat):
    titles_=[]
    marcas_=[]
    image_=[]
    price_=[]
    sub_cat_ = []
    items = item_json
    for i in range(len(items)):
        titles_.append(items[i]['title'].lower()) 
        marcas_.append(get_product_brand(items[i])) #extraer la marca
        image_.append(items[i]['thumbnail'])
        price_.append(items[i]['price'])
        sub_cat_.append(clean_name(item_sub_cat[i]))
    df_products = pd.DataFrame(columns=['nombre','categoria','sub_categoria','marca','precio','imagen','nombre_preproc'])
    df_products['nombre'] = titles_
    df_products['categoria'] = clean_name(item_category)
    #df_products['num_categ'] = dict_categs[item_category]*len(items)
    df_products['sub_categoria'] = sub_cat_
    df_products['marca'] = marcas_
    df_products['precio'] = price_
    df_products['imagen'] = image_
    df_products['nombre_preproc'] = pre_proc_nlp(df_products['nombre'])
    return df_products

## Generar dataframe de los productos

In [ ]:
OFFSETS_LIST= list(range(0,50,50))

In [11]:
category_dict = {}
for category in tqdm(prod_categories): 
    df_cat = obtain_sub_categories(category['id'])
    item_json,item_sub_category = obtain_json_items_iteratively(df_cat,OFFSETS_LIST) #train
    cat_name = category['name']
    df_cat_prod = create_pd_product(item_json,cat_name,item_sub_category)
    category_dict[df_cat_prod['categoria'][0]] = df_cat_prod

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [04:58<00:00,  9.32s/it]


In [12]:
df_dataset = pd.concat([category_dict [cat] for cat in category_dict], axis=0)
df_cats_sub_cats = pd.DataFrame(df_dataset[['sub_categoria','categoria']].value_counts()).reset_index()

In [21]:
df_cats_sub_cats.head(10)

,sub_categoria,categoria,0
0,aberturas,construccion,50
1,otros,arte libreria y merceria,50
2,otros,electrodomesticos y aires ac,50
3,otros,deportes y fitness,50
4,otros,construccion,50
5,otros,consolas y videojuegos,50
6,otros,computacion,50
7,otros,celulares y telefonos,50
8,otros,camaras y accesorios,50
9,otros,belleza y cuidado personal,50


In [24]:
df_sub_registers = pd.DataFrame(df_dataset['sub_categoria'].value_counts()).reset_index()
df_sub_registers

,index,sub_categoria
0,otros,1400
1,proyectores y pantallas,100
2,herramientas industriales,100
3,ropa y calzado para bebes,100
4,motos,100
...,...,...
346,acido clorhidrico,33
347,eventos deportivos,28
348,coberturas extendidas,27
349,eventos a beneficio,15


Se identifica que la sub categoría "otros" está asociada a múltiples categorías
Por lo tanto, es la sub categoría predominante 

### Generar One hot encoding para cada sub categoría y cada categoria

In [13]:
def encoding_categs(categories):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(categories)
    return integer_encoded

In [14]:
numeric_sub_categories = encoding_categs(df_dataset['sub_categoria'].unique())
dict_sub_categs = dict(zip(df_dataset['sub_categoria'].unique(),numeric_sub_categories))
num_list = []
for cat_in in df_dataset['sub_categoria']: 
    num_list.append(dict_sub_categs[cat_in])
df_dataset['numeric_sub_cat'] = num_list

In [25]:
numeric_categories = encoding_categs(df_dataset['categoria'].unique())
dict_categs = dict(zip(df_dataset['categoria'].unique(),numeric_categories))
num_list = []
for cat_in in df_dataset['categoria']: 
    num_list.append(dict_categs[cat_in])
df_dataset['numeric_cat'] = num_list

In [28]:
df_dataset['numeric_cat'].nunique()

32

In [15]:
df_dataset['numeric_sub_cat'].nunique()

351

* En total se cuenta con 351 únicas subcategorías 

### Guardar dataset completo

Se genera el .csv con el nombre dataset_sub_cats, que contiene toda la información para los productos previamente consultados. La dirección para guardar el archivo será la misma donde se encuentre almacenado este notebook

In [29]:
cwd = os.getcwd()
filepath = Path(cwd+'/dataset_sub_cats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_dataset.to_csv(filepath,index=False) 

## Descargar imágenes

In [18]:
def download_images(image_route, category,name_folder):
    for ind,img_url in enumerate(image_route):
        url = img_url
        file_name = category + str(ind) +'.jpg' 
        save_path = './sub_images_'+name_folder+'/'+ category #ruta para guardar las imágenes del dataset
        if ind==0:
            os.makedirs(save_path)
        completeName = os.path.join(save_path, file_name)
        try:
            urllib.request.urlretrieve(url, completeName)
        except: 
            print(ind,category)
            print(url)
            print("problema con url")

In [19]:
for c_ in tqdm(df_dataset['categoria'].unique()):
    download_images(df_dataset[df_dataset['categoria']==c_]['imagen'],c_,'_')

 53%|███████████████████████████████████████████████████████████████████████████████████████▋                                                                             | 17/32 [05:52<06:27, 25.85s/it]

144 entradas para eventos

problema con url
184 entradas para eventos

problema con url
205 entradas para eventos

problema con url


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊     | 31/32 [09:52<00:19, 19.49s/it]

402 otras categorias

problema con url


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [10:07<00:00, 18.98s/it]


# Conclusiones

* Se generó un dataframe, almacenado en un tipo de archivo .csv que contiene toda la información categórica de los productos. Este dataframe contiene un total de 19271 registros, equivalentes al total de productos consultados.

* Adicionalmente, se genera una carpeta (sub_images) que contiene las imágenes de los productos descargados. Esta carpeta está organizada en subcarpetas, lo que significa que las imágenes pertenecientes a cada clase están almacenadas en su respectiva carpeta. 